In [1]:
import numpy as np

In [2]:
class Jaya():
    def __init__(self):
        self.obj = None
        self.population = None
        self.dimension = None
        self.ub = None
        self.lb = None
        self.optimization = "maximize"
        self.max_iter = None
        self.global_best_agent = None
        
    def compile(self,obj,pop_size,dimension,lb,ub,optimization="maximize"):
        self.obj = obj
        self.population_size = pop_size
        self.dimension = dimension
        self.ub = ub
        self.lb = lb
        self.optimization=optimization
        
    def __initialize(self):
        self.population = np.clip(np.random.rand(self.population_size,self.dimension),self.lb,self.ub)
        self.ev_value = np.empty(self.population_size)
        
        for i,value in enumerate(self.population):
            self.ev_value[i] = self.obj(value)
        
    def __update_best_worst(self):
        max_id = np.argmax(self.ev_value)
        min_id = np.argmin(self.ev_value)
        
        if self.optimization == "maximize":
            self.best_agent = (self.population[max_id],self.ev_value[max_id])
            self.worst_agent = (self.population[min_id],self.ev_value[min_id])
        elif self.optimization == "minimize":
            self.worst_agent = (self.population[max_id],self.ev_value[max_id])
            self.best_agent = (self.population[min_id],self.ev_value[min_id])
            
    def __update_global_best(self):
        if self.global_best_agent == None:
            self.global_best_agent = self.best_agent
        elif self.optimization == "maximize" and self.best_agent[1] > self.global_best_agent[1]:
            self.global_best_agent = self.best_agent
        elif self.optimization == "minimize" and self.best_agent[1] < self.global_best_agent[1]:
            self.global_best_agent = self.best_agent
           
    def train(self,max_iter):
        
        self.__initialize()
        self.__update_best_worst()
        for i in range(max_iter):
            
            self.__update_best_worst()
            self.__update_global_best()
            r1 = np.random.rand(self.population_size,self.dimension)
            r2 = np.random.rand(self.population_size,self.dimension)
            population_updated = self.population + r1 * (self.best_agent[0] - np.abs(self.population)) - r2 * (self.worst_agent[0] - np.abs(self.population))
            
            
            
            self.population = np.clip(population_updated,lb,ub)
            
            for i,value in enumerate(self.population):
                self.ev_value[i] = self.obj(value)     
        return self.global_best_agent
            

In [3]:
from benchmarking.benchmark import *
function=[Sphere,Schwefel_2_22,Schwefel_1_2,Rosenbrock,Step,Quartic_with_noise,Ackley,SumSquare]


In [5]:
for i in function:
    func=i()
    values=np.empty(30)
    for l in range(30):
        lb = np.array([func.range[0]]).repeat(func.dim)
        ub = np.array([func.range[1]]).repeat(func.dim)
        jaya = Jaya()
        jaya.compile(func.get_algorithm(),pop_size=100,dimension=func.dim,lb=lb,ub=ub,optimization = "minimize")
        values[l]=jaya.train(100)[-1]
    print(f"{func.name}\t ==> Min = {np.min(values)} Max = {np.max(values)} Mean = {np.mean(values)}")

Sphere	 ==> Min = 2.675878513351252 Max = 5.999913956402348 Mean = 4.256207176084625
Schwefel_2_22	 ==> Min = -9.999999999999999e+29 Max = -1.2278587720525254e+29 Mean = -5.191988759103761e+29
Schwefel_1_2	 ==> Min = 78.12023839995122 Max = 604.0510075984497 Mean = 208.70670105349356
Rosenbrock	 ==> Min = 290.51799128207296 Max = 396.68468422067656 Mean = 336.6324630285438
Step	 ==> Min = 8.092118427699807 Max = 16.23563751227735 Mean = 11.197343365838721
Quartic_with_noise	 ==> Min = 22.809942513658214 Max = 55.54994751740249 Mean = 35.60112661137922
Ackley	 ==> Min = 2.7036659088179964 Max = 3.3724744310274812 Mean = 3.051022665817203
SumSquare	 ==> Min = 46.49018976688197 Max = 78.77051985830745 Mean = 59.59225876499098
